In [1]:
!pip install lingam

In [2]:
#German
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.model_selection import train_test_split

from csse import CSSE
from prepare_dataset import *

import pickle

import numpy as np
import pandas as pd
import graphviz
import lingam
from lingam.utils import print_causal_directions, print_dagc, make_dot

from tqdm import tqdm
import random as rnd

from IPython.display import display

import warnings

warnings.filterwarnings('ignore')

In [4]:
map_columns = {
    'Unnamed: 0': 'index',
    'x0': 'Sex',
    'x1': 'Age',
    'x2': 'Credit',
    'x3': 'LoanDuration'
}

In [11]:
# Read Dataset German
x_train = pd.read_csv("data/algrec_german/X_train.csv").rename(columns=map_columns)
x_test = pd.read_csv("data/algrec_german/X_test.csv").rename(columns=map_columns)
y_train = pd.read_csv("data/algrec_german/y_train.csv").rename(columns={'Unnamed: 0': 'index'})
y_test = pd.read_csv("data/algrec_german/y_test.csv").rename(columns={'Unnamed: 0': 'index'})

x_train = x_train.set_index('index')
x_test = x_test.set_index('index')
y_train = y_train.set_index('index')
y_test = y_test.set_index('index')

dfx_full = pd.concat([x_train, x_test])
dfy_full = pd.concat([y_train, y_test])

In [66]:
model = RandomForestClassifier()  
model.fit(x_train, y_train)

index_samples = [747, 349, 646, 76, 890]

x_test_sample = x_test.loc[index_samples]
y_test_sample = y_test.loc[index_samples]


p = model.predict(x_test)

print(classification_report(y_test, p))

K = 1 #Number of counterfactual explanations to be obtained

explainerCSSE = CSSE(dfx_full, model, K = K, num_gen = 10)

              precision    recall  f1-score   support

         0.0       0.47      0.62      0.53        66
         1.0       0.60      0.44      0.51        84

    accuracy                           0.52       150
   macro avg       0.53      0.53      0.52       150
weighted avg       0.54      0.52      0.52       150



In [88]:
model_lingam = lingam.DirectLiNGAM()

result_lingam_bt = model_lingam.bootstrap(dfx_full, n_sampling=500)

causal_effects = result_lingam_bt.get_total_causal_effects(min_causal_effect=0.01)

df_causal_effects = pd.DataFrame(causal_effects)
labels = [f'{i}' for i in dfx_full.columns]

df_causal_effects['from'] = df_causal_effects['from'].apply(lambda x : labels[x])
df_causal_effects['to'] = df_causal_effects['to'].apply(lambda x : labels[x])

causal_order = [labels[x] for x in model_lingam.causal_order_]

In [164]:
causal_order

['Sex', 'LoanDuration', 'Age', 'Credit']

In [89]:
df_causal_effects

,from,to,effect,probability
0,Sex,Age,5.552540,0.994
1,LoanDuration,Credit,137.330951,0.652
2,Sex,Credit,543.266280,0.112
3,LoanDuration,Age,-0.069263,0.106
4,Sex,LoanDuration,2.815671,0.098
5,Age,LoanDuration,-0.112837,0.036
6,Age,Credit,24.997032,0.028


In [68]:
df_causal_effects

,from,to,effect,probability
0,Sex,Age,5.729192,0.99
1,LoanDuration,Credit,137.503368,0.79
2,Sex,Credit,164.961538,0.07
3,LoanDuration,Age,-0.001102,0.06
4,Age,Credit,26.430504,0.04
5,Sex,LoanDuration,0.616003,0.03
6,Age,LoanDuration,0.009957,0.02


In [172]:
def apply_causality(df):
#     print(f'df len = {df.shape}')
    df_apply_causal = pd.DataFrame(columns = df.columns)
    original = df.iloc[0]
    df_apply_causal.loc[0] = original
    for index, df_row in df.iloc[1:].iterrows():
        causal_ind = df_row.copy()
        for column in causal_order:
            value_diff = causal_ind[column] - original[column]
#             print(f"value_diff = {value_diff}")
            if value_diff != 0:
                tmp_effects = df_causal_effects[df_causal_effects['from'] == column]
                for index, row in tmp_effects.iterrows():
#                     prob = rnd.random()
#                     if row['probability'] <= prob:
#                     print(f"valor antes do efeito = {causal_ind[row['to']]}")
                    causal_ind[row['to']] = causal_ind[row['to']] + (value_diff * row['effect'])
#                     print(f"valor depois do efeito = {causal_ind[row['to']]}")
        df_apply_causal.loc[len(df_apply_causal)] = causal_ind
    return df_apply_causal
    
def euclidean_distance(series1, series2):
    # Verificando se as séries têm o mesmo comprimento
    if len(series1) != len(series2):
        raise ValueError("As séries devem ter o mesmo comprimento.")
    
    # Calculando a diferença quadrática entre os elementos das séries
    squared_diff = (series1 - series2) ** 2
    
    # Calculando a soma das diferenças quadráticas
    sum_squared_diff = np.sum(squared_diff)
    
    # Calculando a raiz quadrada da soma
    euclidean_dist = np.sqrt(sum_squared_diff)
    
    return euclidean_dist

def get_contrafac_df_causal(solution_list_causal):
    print(f'solution_list_causal = \n {solution_list_causal}')
    lista_solution_causal = [[t.column for t in sublist] for sublist in solution_list_causal]

    # Inicializa uma lista para armazenar os resultados
    resultados = []

    # Loop sobre os valores na lista
    for lista_valores in lista_solution_causal:
#         print(f'lista_valores = {lista_valores}')
        if len(lista_valores) > 1:
            for v1 in lista_valores:
                for v2 in lista_valores:
                    if v1 != v2:
                        # Cria uma condição para cada par de valores diferentes na lista
                        condicao = (df_causal_effects['to'].isin([v1, v2])) & (df_causal_effects['from'].isin([v1, v2]))
#                         condicao = (df_causal_effects['to'].isin([v1, v2])) | (df_causal_effects['from'].isin([v1, v2]))
                        # Realiza a busca no DataFrame usando a condição e armazena os resultados
                        resultados.append(df_causal_effects[condicao])

    # Concatena os resultados em um único DataFrame
    if resultados:
        resultado_final = pd.concat(resultados)
        resultado_final = resultado_final.drop_duplicates().sort_values(by='probability')
    else:
        resultado_final = pd.DataFrame(columns = df_causal_effects.columns)
        
    return resultado_final

def analyse_contrafac(contrafac, df, original_ind):
#     if df.empty:
#         return []
    columns = [x.column for x in contrafac]
#     print(f'columns analyse contrafac = {columns}')
    condicao = (df['to'].isin(columns)) & (df['from'].isin(columns))
#     condicao = (df['to'].isin(columns)) | (df['from'].isin(columns))
    ind = original_ind[columns]
    return [contrafac, df[condicao], ind]

def verificar_condicoes(row):
    if (row['from'] == 'mais' and row['to'] == 'mais' and row['effect'] > 0):
        return True
    elif row['from'] == 'menos' and row['to'] == 'menos' and row['effect'] > 0:
        return True
    elif row['from'] == 'mais' and row['to'] == 'menos' and row['effect'] < 0:
        return True
    elif row['from'] == 'menos' and row['to'] == 'mais' and row['effect'] < 0:
        return True
    else:
        return False

In [175]:
def get_causal_explain(X):
    original_ind = x_test.iloc[X].copy() #Original instance
    #self.ind_cur_class = ind_cur_class #Index in the shap corresponds to the original instance class
    explainerCSSE.current_class = p[X] #Original instance class
    explainerCSSE.original_ind = original_ind

    ind_cur_class = explainerCSSE.getBadClass()

    #Gets the valid values range of each feature
    features_range = []
    features_range = explainerCSSE.getFeaturesRange()

    #The DataFrame df will have the current population
    df = pd.DataFrame(columns=explainerCSSE.input_dataset.columns)

    #Generates the initial population with popinitial mutants        
    explainerCSSE.getPopInicial(df, features_range)
    df_causal = df.copy()
    reut
    dict_dfs = {}

    # for g in tqdm(range(explainerCSSE.num_gen), desc= "Processing..."):
    for g in tqdm(range(10), desc= "Processing..."):

        #To use on the parents of each generation
        parents = pd.DataFrame(columns=explainerCSSE.input_dataset.columns)

        #Copy parents to the next generation
#         parents = df.copy()
        parents_causal = df_causal.copy()
    #     print(f"parents = {len(parents)}")
        dict_dfs[g] = {}

#         dict_dfs[g]['original_parents'] = parents
        dict_dfs[g]['causal_parents'] = apply_causality(parents_causal)
    #     raise KeyboardInterrupt("Execução interrompida pelo usuário")
        #df will contain the new population
#         df = pd.DataFrame(columns=explainerCSSE.input_dataset.columns)
        df_causal = pd.DataFrame(columns=explainerCSSE.input_dataset.columns)
#         evaluation = []
        evaluation_causal = []

        #Assessing generation counterfactuals
#         explainerCSSE.fitness(parents, evaluation, ind_cur_class)
        explainerCSSE.fitness(dict_dfs[g]['causal_parents'], evaluation_causal, ind_cur_class)

    #     print(len(df))
    #     dict_dfs[g]['original'] = df.copy()
    #     df_causal_applied = apply_causality(df)
    #     dict_dfs[g]['causal'] = df_causal_applied
    #     print(f"df depois do fitness = {len(df)}")
    #     raise KeyboardInterrupt("Execução interrompida pelo usuário")


        #The original individual will always be in the 0 position of the df - So that it is normalized too (it will be used later in the distance function)
#         df.loc[0] = original_ind.copy()
        df_causal.loc[0] = original_ind.copy()

        #Copies to the next generation the per_elit best individuals
#         explainerCSSE.elitism(evaluation, df, parents)
        explainerCSSE.elitism(evaluation_causal, df_causal, parents_causal)
    #     raise KeyboardInterrupt("Execução interrompida pelo usuário")
    #     print(f"df depois do elitism = {len(df)}")
        number_cross_repetitions = 0
        while len(df_causal) < explainerCSSE.pop_size + 1: #+1, as the 1st position is used to store the reference individual
#             number_cross_repetitions = explainerCSSE.crossover(df, parents, evaluation, number_cross_repetitions)
            number_cross_repetitions_causal = explainerCSSE.crossover(df_causal, parents_causal, evaluation_causal, number_cross_repetitions)

            mutation_op = rnd.random()
            if mutation_op <= explainerCSSE.mutation_proba:
#                 explainerCSSE.mutation(df, len(df) - 1, features_range)
                explainerCSSE.mutation(df_causal, len(df_causal) - 1, features_range)
    #     raise KeyboardInterrupt("Execução interrompida pelo usuário")

    # raise KeyboardInterrupt("Execução interrompida pelo usuário")

    evaluation = []
    evaluation_causal = []

    #Evaluating the latest generation
#     explainerCSSE.fitness(df, evaluation, ind_cur_class)
    explainerCSSE.fitness(df_causal, evaluation_causal, ind_cur_class)

    #Order the last generation by distance to the original instance     
#     evaluation.sort(key=lambda individual: individual.aval_norm)
    evaluation_causal.sort(key=lambda individual: individual.aval_norm) 

    #Getting the counterfactual set
#     contrafactual_set = pd.DataFrame(columns=explainerCSSE.input_dataset.columns)
#     contrafactual_set, solution_list = explainerCSSE.getContrafactual(df, evaluation)

    #Getting the counterfactual CAUSAL set
#     contrafactual_set_causal = pd.DataFrame(columns=explainerCSSE.input_dataset.columns)
    print('df_causal para achar contrafac = ')
    display(df_causal)
    print('\n evaluation')
    print(evaluation_causal)
    contrafactual_set_causal, solution_list_causal = explainerCSSE.getContrafactual(df_causal, evaluation_causal) 

    dict_dfs['contrafactual_set_causal'] = contrafactual_set_causal
    dict_dfs['solution_list_causal'] = solution_list_causal
#     print(f'solution_list_causal = {solution_list_causal}')
    df_contrafac_causal = get_contrafac_df_causal(solution_list_causal)
    return [dict_dfs, df_contrafac_causal, solution_list_causal]
    return [solution_list_causal, solution_list, df_contrafac_causal, original_ind]

In [124]:
def run(original_row):
    response_list = get_causal_explain(original_row)
#     print(response_list[0], '\n', response_list[1], '\n', response_list[2], '\n')
#     print(f"solution_list_causal = {response_list[0]}\n")
#     print(f"solution_list = {response_list[1]}\n")
#     print(f"df_contrafac_causal = {response_list[2]}\n")
#     print(f"original_ind = {response_list[3]}\n")
#     raise KeyboardInterrupt
    return response_list
    list_analyse = []
    for contrafac_causal in response_list[0]:
        list_analyse.append(analyse_contrafac(contrafac_causal, response_list[2], response_list[3]))
#     print(f"tamanho da list_analyse = {len(list_analyse)}")
    return list_analyse

In [110]:
%time
global_quant_changes = 0
global_quant_causal_changes = 0
global_quant_causal_rules = 0
global_quant_zeros_causal = 0
global_quant_full_causal = 0
global_quant_causal_contrafac = 0

quant_original_instance = 10
for x in range(quant_original_instance):
    print(f"\nrun {x}")
    runs = run(x)
    for content in runs:
        controle = {}
        causal = content[0]
        df = content[1]
        ori = content[2]
        
        num_changes = len(causal)
        global_quant_changes += num_changes
        
        num_causal_rules = len(df)
        global_quant_causal_rules += num_causal_rules
        
        for attr in causal:
            key = attr.column
            if attr.value > ori[key]:
                controle[key] = 'mais'
            else:
                controle[key] = 'menos'

        df_temp = df.copy()
        df_temp['from'] = df['from'].map(controle)
        df_temp['to'] = df['to'].map(controle)
        df_temp['causal'] = df_temp.apply(verificar_condicoes, axis = 1)
        
        causal_finds = df_temp['causal'].sum()
        global_quant_causal_changes += causal_finds

        if causal_finds > 0:
            global_quant_causal_contrafac += 1
        else:
            global_quant_zeros_causal += 1
            display(df_temp)
            print(f"original = {ori}")
            print(f"causal = {causal}")
        
        if causal_finds == num_causal_rules:
            global_quant_full_causal += 1
            if causal_finds > 2:
                display(df_temp)
                print(f"original = {ori}")
                print(f"causal = {causal}")

                
quant_contrafac = quant_original_instance*K

print()
print(f"quantas instancias contrafactuais encontradas = {quant_contrafac}")
print(f"quantidade de mudanças totais = {global_quant_changes}")

print(f"quantas instancias tiveram pelo menos uma relação causal satisfeita = {global_quant_causal_contrafac}/{quant_contrafac}")

print(f"quantas relações causais foram encontradas = {global_quant_causal_rules}")

print(f"quantas relações causais foram satisfeitas = {global_quant_causal_changes}/{global_quant_causal_rules}")

print(f"quantas instâncias não tiveram nenhuma relação causal satisfeita = {global_quant_zeros_causal}/{quant_contrafac}")
print(f"quantas instâncias tiveram TODAS as relaçoes causais satisfeitas = {global_quant_full_causal}/{global_quant_causal_contrafac}")

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs

run 0


Processing...: 100%|██████████| 10/10 [00:19<00:00,  1.91s/it]

[[[('Credit', 792.97)]], [[('Credit', 792.97)]], Empty DataFrame
Columns: [from, to, effect, probability]
Index: [], Sex                0.0
Age               37.0
Credit          1274.0
LoanDuration      12.0
Name: 747, dtype: float64]
tamanho da list_analyse = 1


,from,to,effect,probability,causal


original = Credit    1274.0
Name: 747, dtype: float64
causal = [('Credit', 792.97)]

run 1


...

original = LoanDuration    42.0
Name: 76, dtype: float64
causal = [('LoanDuration', 43.59)]

run 9


Processing...: 100%|██████████| 10/10 [00:20<00:00,  2.00s/it]

[[[('Credit', 2312.96)]], [[('Credit', 2312.96)]], Empty DataFrame
Columns: [from, to, effect, probability]
Index: [], Sex                1.0
Age               27.0
Credit          2613.0
LoanDuration      36.0
Name: 225, dtype: float64]
tamanho da list_analyse = 1


,from,to,effect,probability,causal


original = Credit    2613.0
Name: 225, dtype: float64
causal = [('Credit', 2312.96)]

quantas instancias contrafactuais encontradas = 10
quantidade de mudanças totais = 11
quantas instancias tiveram pelo menos uma relação causal satisfeita = 0/10
quantas relações causais foram encontradas = 1
quantas relações causais foram satisfeitas = 0.0/1
quantas instâncias não tiveram nenhuma relação causal satisfeita = 10/10
quantas instâncias tiveram TODAS as relaçoes causais satisfeitas = 9/0


In [104]:
for x in range(10):
    print(f"run {x}")
    runs = run(x)

run 0


Processing...: 100%|██████████| 10/10 [00:25<00:00,  2.60s/it]


solution_list_causal = [[('Credit', 765.0)]]
lista_valores = ['Credit']
[[[('Credit', 765.0)]], [[('Credit', 394.49)]], Empty DataFrame
Columns: [from, to, effect, probability]
Index: [], Sex                0.0
Age               37.0
Credit          1274.0
LoanDuration      12.0
Name: 747, dtype: float64]
columns analyse contrafac = ['Credit']
tamanho da list_analyse = 1
run 1


Processing...: 100%|██████████| 10/10 [00:26<00:00,  2.63s/it]


solution_list_causal = [[('LoanDuration', 7.24)]]
lista_valores = ['LoanDuration']
[[[('LoanDuration', 7.24)]], [[('LoanDuration', 7.24)]], Empty DataFrame
Columns: [from, to, effect, probability]
Index: [], Sex                1.0
Age               32.0
Credit          1136.0
LoanDuration       9.0
Name: 349, dtype: float64]
columns analyse contrafac = ['LoanDuration']
tamanho da list_analyse = 1
run 2


...

solution_list_causal = [[('Credit', 811.38)]]
lista_valores = ['Credit']
[[[('Credit', 811.38)]], [[('LoanDuration', 17.39)]], Empty DataFrame
Columns: [from, to, effect, probability]
Index: [], Sex               1.0
Age              41.0
Credit          719.0
LoanDuration     12.0
Name: 443, dtype: float64]
columns analyse contrafac = ['Credit']
tamanho da list_analyse = 1
run 8


Processing...: 100%|██████████| 10/10 [00:26<00:00,  2.60s/it]


solution_list_causal = [[('Age', 30.88)]]
lista_valores = ['Age']
[[[('Age', 30.88)]], [[('LoanDuration', 44.27)]], Empty DataFrame
Columns: [from, to, effect, probability]
Index: [], Sex                1.0
Age               34.0
Credit          3965.0
LoanDuration      42.0
Name: 76, dtype: float64]
columns analyse contrafac = ['Age']
tamanho da list_analyse = 1
run 9


Processing...: 100%|██████████| 10/10 [00:25<00:00,  2.56s/it]

solution_list_causal = [[('Age', 26.06)]]
lista_valores = ['Age']
[[[('Age', 26.06)]], [[('Age', 26.06)]], Empty DataFrame
Columns: [from, to, effect, probability]
Index: [], Sex                1.0
Age               27.0
Credit          2613.0
LoanDuration      36.0
Name: 225, dtype: float64]
columns analyse contrafac = ['Age']
tamanho da list_analyse = 1


In [122]:
for x in range(10):
    print(f"run {x}")
    runs = run(x)

run 0


Processing...:   0%|          | 0/10 [00:00<?, ?it/s]

valor antes do efeito = 37.0
valor depois do efeito = 38.998914241687906
valor antes do efeito = 1274.0
valor depois do efeito = 1469.5758607712016
valor antes do efeito = 12.0
valor depois do efeito = 13.01364138623807
valor antes do efeito = 1469.5758607712016
valor depois do efeito = 1608.780196098423
valor antes do efeito = 38.998914241687906
valor depois do efeito = 38.92870593222768
valor antes do efeito = 13.01364138623807
valor depois do efeito = 12.796012482937511
valor antes do efeito = 1608.780196098423
valor depois do efeito = 1656.992120705661
valor antes do efeito = 12.0
valor depois do efeito = 11.215784608391855
valor antes do efeito = 1274.0
valor depois do efeito = 1447.7293749251298
valor antes do efeito = 12.0
valor depois do efeito = 8.3576298070344
valor antes do efeito = 1274.0
valor depois do efeito = 2080.9042046882287
valor antes do efeito = 1274.0
valor depois do efeito = 5943.252326694079
valor antes do efeito = 37.0
valor depois do efeito = 34.6450423650251

valor antes do efeito = 1616.257756349603
valor depois do efeito = 1859.8653431722403
valor antes do efeito = 40.498099922953834
valor depois do efeito = 40.37523538139843
valor antes do efeito = 13.773872425916625
valor depois do efeito = 13.393021845140648
valor antes do efeito = 1859.8653431722403
valor depois do efeito = 1944.2362112349067
valor antes do efeito = 37.0
valor depois do efeito = 37.777355538434186
valor antes do efeito = 1274.0
valor depois do efeito = 1350.0572791888005
valor antes do efeito = 12.0
valor depois do efeito = 12.394193872425916
valor antes do efeito = 1350.0572791888005
valor depois do efeito = 1404.19229848272
valor antes do efeito = 37.777355538434186
valor depois do efeito = 37.75005230697743
valor antes do efeito = 12.394193872425916
valor depois do efeito = 12.309560410031255
valor antes do efeito = 1404.19229848272
valor depois do efeito = 1422.9413802744236
valor antes do efeito = 1274.0
valor depois do efeito = 6509.055843928772
valor antes do e

Processing...:  10%|█         | 1/10 [00:02<00:22,  2.51s/it]

...

valor antes do efeito = 34.0
valor depois do efeito = 31.77898417590233
valor antes do efeito = 3965.0
valor depois do efeito = 3747.693488031998
valor antes do efeito = 42.0
valor depois do efeito = 40.873731793068806
valor antes do efeito = 3747.693488031998
valor depois do efeito = 3593.022004335085
valor antes do efeito = 31.77898417590233
valor depois do efeito = 31.85699340863592
valor antes do efeito = 40.873731793068806
valor depois do efeito = 41.11554168562498
valor antes do efeito = 3593.022004335085
valor depois do efeito = 3539.453199215932
valor antes do efeito = 42.0
valor depois do efeito = 41.9943581626503
valor antes do efeito = 3965.0
valor depois do efeito = 3966.2498516181663
valor antes do efeito = 42.0
valor depois do efeito = 39.97570875892804
valor antes do efeito = 3965.0
valor depois do efeito = 4413.446760598104
valor antes do efeito = 42.0
valor depois do efeito = 37.60613707205451
valor antes do efeito = 3965.0
valor depois do efeito = 4938.384440227993
va

Processing...:  10%|█         | 1/10 [00:02<00:19,  2.13s/it]

valor antes do efeito = 42.0
valor depois do efeito = 42.409597391588136
valor antes do efeito = 3965.0
valor depois do efeito = 3874.260772521119
valor antes do efeito = 3965.0
valor depois do efeito = 4640.66827786279
valor antes do efeito = 34.0
valor depois do efeito = 33.659223777527174
valor antes do efeito = 46.92
valor depois do efeito = 46.95845208039673
valor antes do efeito = 4640.66827786279
valor depois do efeito = 4632.149883600984
valor antes do efeito = 34.0
valor depois do efeito = 30.779527055058377
valor antes do efeito = 3965.0
valor depois do efeito = 3649.9055576463975
valor antes do efeito = 42.0
valor depois do efeito = 40.366911099949775
valor antes do efeito = 3649.9055576463975
valor depois do efeito = 3425.6319062858743
valor antes do efeito = 30.779527055058377
valor depois do efeito = 30.89264044252208
valor antes do efeito = 40.366911099949775
valor depois do efeito = 40.717535444156226
valor antes do efeito = 3425.6319062858743
valor depois do efeito = 3

Processing...:  10%|█         | 1/10 [00:02<00:23,  2.65s/it]


KeyboardInterrupt: 

In [177]:
response_run = run(3)

Processing...: 100%|██████████| 10/10 [00:12<00:00,  1.26s/it]

df_causal para achar contrafac = 


,Sex,Age,Credit,LoanDuration
0,1.00,46.00,697.00,12.00
1,1.00,46.00,5022.50,12.00
2,1.00,46.00,532.26,18.00
3,1.00,46.00,697.00,18.22
4,1.00,33.49,1566.78,12.00
...,...,...,...,...
77,1.00,38.37,14047.92,58.24
78,0.76,56.80,532.26,18.00
79,0.63,46.00,11213.92,12.00
80,0.01,72.30,697.00,12.00



 evaluation
[(3, 0, 0.0980453972257251, 1, 0.32199610690081343, 0.32199610690081343, 0.44), (1, 0, 0.2495862828232937, 1, 0.43327469934924945, 0.43327469934924945, 0.48), (69, 0, 0.2642348713379971, 1, 0.4440313628387407, 0.4440313628387407, 0.42), (15, 0, 0.3295485231970125, 1, 0.4919920911786773, 0.4919920911786773, 0.34), (20, 0, 0.3461878010557001, 1, 0.5042105457044762, 0.5042105457044762, 0.37), (29, 0, 0.39987698116865583, 1, 0.5436352618074691, 0.5436352618074691, 0.22), (67, 0, 0.4237289015780108, 1, 0.5611500606677164, 0.5611500606677164, 0.17), (2, 0, 0.0950540462457132, 2, 0.5697995160252728, 0.5697995160252728, 0.44), (44, 0, 0.45964691046658257, 1, 0.5875251571105657, 0.5875251571105657, 0.46), (55, 0, 0.4597944223769638, 1, 0.5876334771701449, 0.5876334771701449, 0.16), (7, 0, 0.5138713745271123, 1, 0.6273429397052279, 0.6273429397052279, 0.37), (52, 0, 0.17947636775255496, 2, 0.6317920078301471, 0.6317920078301471, 0.47), (33, 0, 0.5205231416193709, 1, 0.63222742534355

In [137]:
response_run[0][9]

{'original_parents':      Sex    Age    Credit  LoanDuration
 0   0.00  37.00   1274.00         12.00
 1   0.00  37.00   1502.12         12.00
 2   0.00  44.63   1274.00         12.00
 3   0.87  37.00   1274.00          4.14
 4   0.03  34.12   1545.52         12.00
 ..   ...    ...       ...           ...
 77  0.03  58.13   3746.53         12.00
 78  0.00  45.41  17017.15         12.00
 79  0.00  37.00   4990.48         12.00
 80  0.00  22.45   4732.42         12.00
 81  0.41  45.41   4384.61         55.45
 
 [82 rows x 4 columns],
 'causal_parents':      Sex        Age       Credit  LoanDuration
 0   0.00  37.000000  1274.000000     12.000000
 1   0.00  37.000000  1989.800000     12.000000
 2   0.00  38.200000  1303.996439     11.864596
 3   0.00  37.083116  1111.280516     10.790621
 4   0.03  37.160725  1305.916010     12.066334
 ..   ...        ...          ...           ...
 59  0.00  36.824935  3597.924748     11.659754
 60  0.00  28.636069  4277.705699     27.293759
 61  0.00  3

In [138]:
response_run[0]

{0: {'original_parents':     Sex    Age    Credit  LoanDuration
  0   0.0  37.00   1274.00         12.00
  1   0.0  37.00  14862.98         12.00
  2   0.0  37.00   4987.72         12.00
  3   0.0  32.78   1274.00         12.00
  4   0.0  54.31   1274.00         12.00
  ..  ...    ...       ...           ...
  77  0.0  37.00   4990.48         12.00
  78  0.0  24.06   1274.00         12.00
  79  0.0  37.00  13083.16         12.00
  80  0.0  37.00   1274.00         71.15
  81  0.0  37.00   1274.00         63.38
  
  [82 rows x 4 columns],
  'causal_parents':     Sex        Age        Credit  LoanDuration
  0   0.0  37.000000   1274.000000     12.000000
  1   0.0  37.000000  14862.980000     12.000000
  2   0.0  37.000000   4987.720000     12.000000
  3   0.0  32.780000   1168.512523     12.476171
  4   0.0  54.310000   1706.698630     10.046796
  ..  ...        ...           ...           ...
  77  0.0  37.000000   4990.480000     12.000000
  78  0.0  24.060000    950.538401     13.46010

In [150]:
x_test.iloc[3]

Sex               1.0
Age              46.0
Credit          697.0
LoanDuration     12.0
Name: 925, dtype: float64

In [153]:
response_run[0]['contrafactual_set_causal']

,Sex,Age,Credit,LoanDuration
0,1.0,46.0,697.0,17.23


In [154]:
response_run[0]['solution_list_causal']

[[('LoanDuration', 17.23)]]

In [161]:
zxc = response_run[0][9]['causal_parents']
zxc

,Sex,Age,Credit,LoanDuration
2,1.00,45.637752,1406.185750,17.270875
6,1.00,45.054554,7834.084128,25.756681
8,0.93,25.979006,5020.142765,24.832007
12,1.00,44.688150,3265.255853,31.088025
15,1.00,42.869984,13265.964561,57.543181
16,1.00,45.596887,1486.189496,17.865486
20,1.00,39.906887,1343.956382,18.507527
21,1.00,63.844134,10498.698229,45.876526
23,1.00,45.640197,10198.300472,50.730599
25,0.74,48.009148,23161.923538,50.281220


In [171]:
zxc

,Sex,Age,Credit,LoanDuration
0,1.00,46.000000,697.000000,12.000000
1,1.00,46.000000,5123.840000,12.000000
2,1.00,45.637752,1406.185750,17.270875
3,1.00,34.330000,405.284632,13.316805
4,0.74,44.330685,961.436863,15.446286
...,...,...,...,...
77,0.06,41.403757,5755.097750,3.521895
78,0.44,43.849775,16646.157229,55.855849
79,1.00,73.440000,3021.058568,8.903760
80,1.00,44.301082,7073.779143,12.031700


In [183]:
asd = zxc.iloc[[0,1,2,3,4]]
asd

,Sex,Age,Credit,LoanDuration
0,1.00,46.000000,697.000000,12.000000
1,1.00,46.000000,5123.840000,12.000000
2,1.00,45.637752,1406.185750,17.270875
3,1.00,34.330000,405.284632,13.316805
4,0.74,44.330685,961.436863,15.446286


In [184]:
qwe = apply_causality(asd)
qwe

,Sex,Age,Credit,LoanDuration
0,1.00,46.000000,697.000000,12.000000
1,1.00,46.000000,5123.840000,12.000000
2,1.00,45.272673,2111.858993,17.352944
3,1.00,34.238794,292.127434,14.643901
4,0.74,42.699029,1110.418381,15.086682


In [ ]:
def apply_causality(df):
#     print(f'df len = {df.shape}')
    df_apply_causal = pd.DataFrame(columns = df.columns)
    original = df.iloc[0]
    df_apply_causal.loc[0] = original
    for index, df_row in df.iloc[1:].iterrows():
        causal_ind = df_row.copy()
        print(f'causal_ind \n {causal_ind}')
        for column in causal_order:
            value_diff = causal_ind[column] - original[column]
#             print(f"value_diff = {value_diff}")
            if value_diff != 0:
                tmp_effects = df_causal_effects[df_causal_effects['from'] == column]
                for index, row in tmp_effects.iterrows():
#                     prob = rnd.random()
#                     if row['probability'] <= prob:
#                     print(f"valor antes do efeito = {causal_ind[row['to']]}")
                    causal_ind[row['to']] = causal_ind[row['to']] + (value_diff * row['effect'])
#                     print(f"valor depois do efeito = {causal_ind[row['to']]}")
        df_apply_causal.loc[len(df_apply_causal)] = causal_ind
    return df_apply_causal

In [156]:
df_causal_effects

,from,to,effect,probability
0,Sex,Age,5.552540,0.994
1,LoanDuration,Credit,137.330951,0.652
2,Sex,Credit,543.266280,0.112
3,LoanDuration,Age,-0.069263,0.106
4,Sex,LoanDuration,2.815671,0.098
5,Age,LoanDuration,-0.112837,0.036
6,Age,Credit,24.997032,0.028


In [178]:
causal_order

['Sex', 'LoanDuration', 'Age', 'Credit']